# Install dependencies if not installed

( !pip install selenium webdriver-manager pandas )

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time


#### Configure Selenium WebDriver

In [4]:

options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Edge(options=options)

base_url = "https://courses.monoprix.tn/lac/2-tous-les-produits"

save_directory = r"C:\Users\abdel\OneDrive\Desktop\w\monoprix_products"
os.makedirs(save_directory, exist_ok=True)


# List products category links to scrape

In [ ]:

category_links = {
    "Epicerie Sucrée": "https://courses.monoprix.tn/lac/2-tous-les-produits?q=Cat%C3%A9gories-Epicerie+Sucr%C3%A9e",
    "Boissons": "https://courses.monoprix.tn/lac/2-tous-les-produits?q=Cat%C3%A9gories-Boissons",
    "Crèmerie et surgelé": "https://courses.monoprix.tn/lac/2-tous-les-produits?q=Cat%C3%A9gories-Cr%C3%A8merie+et+surgel%C3%A9",
    "Hygiène":"https://courses.monoprix.tn/lac/2-tous-les-produits?q=Cat%C3%A9gories-Hygi%C3%A8ne+%26+Beaut%C3%A9",
    "Epicerie Salée": "https://courses.monoprix.tn/lac/2-tous-les-produits?q=Cat%C3%A9gories-Epicerie+Sal%C3%A9e",
}


#### Loop through each category

In [ ]:


for category, link in category_links.items():
    print(f"📌 Scraping Category: {category} ({link})")
    driver.get(link)

    product_data = []  

    while True:
       
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "js-product-list"))
            )
        except:
            print(f"⚠️ Skipping category {category} due to loading issue.")
            break

       
        soup = BeautifulSoup(driver.page_source, "html.parser")

     
        product_section = soup.find("div", id="js-product-list")
        products = product_section.find_all("div", class_="product-miniature") if product_section else []

        for product in products:
            name_tag = product.select_one(".product-title a")
            price_tag = product.select_one(".price")
            image_tag = product.select_one(".product-thumbnail img")

            name = name_tag.text.strip() if name_tag else "N/A"
            price = price_tag.text.strip() if price_tag else "N/A"
            image = image_tag["src"] if image_tag else "N/A"
            product_link = urljoin(base_url, name_tag["href"]) if name_tag else "N/A"

            product_data.append({
                "Category": category,
                "Name": name,
                "Price": price,
                "Image": image,
                "Product Link": product_link
            })

       
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, ".next.js-search-link")
            if "disabled" in next_button.get_attribute("class"):
                break  
            next_button.click()
            time.sleep(2) 
        except:
            break 

    # Save category products to CSV
    if product_data:
        df = pd.DataFrame(product_data)
        category_filename = f"{save_directory}/{category.replace(' ', '_')}.csv"
        df.to_csv(category_filename, index=False, encoding="utf-8-sig")
        print(f"✅ Saved {category} products to: {category_filename}")
    else:
        print(f"⚠️ No products found for {category}, skipping CSV save.")


driver.quit()

print("\n✅ Scraping completed! All categories saved separately.")


📌 Scraping Category: Epicerie Sucrée (https://courses.monoprix.tn/lac/2-tous-les-produits?q=Cat%C3%A9gories-Epicerie+Sucr%C3%A9e)
